# US Visas

In [ ]:
import pandas as pd
from IPython.display import display, Markdown
import matplotlib.pyplot as plt

## Read CSV

In [ ]:
visas_df = pd.read_csv("data/us_perm_visas.csv")

## Display data types

In [ ]:
visas_df.info(verbose=True)

## Count of columns

In [ ]:
print(len(visas_df.columns))

## Print infos

In [ ]:
def print_info_of_str_column(column_object):
    print('###################################')
    print(column_object)
    print('')
    #display(Markdown("# " + column_object))
    print((visas_df[column_object].value_counts()))
    print('###################################')
    print('')

In [ ]:
headers = list(visas_df)
for x in list(visas_df):
    print_info_of_str_column(headers.pop(0))


## Merge Columns

In this case: case_no, case_number

In [ ]:
cols = [['case_no', 'case_number']]

Values are getting concatenated.

In [ ]:
visas_df['Case_No_merged'] = visas_df['case_no'].fillna('') + visas_df['case_number'].fillna('')

Check if there are duplicates.

In [ ]:
visas_df['Case_No_merged'].value_counts()

Duplicate Values are existing - CaseNo is no key value.

Check the length of all concatenated strings. If the merging process was succesful, there should be no length over 13.
Also der should be no NaN or Null values.

In [ ]:
visas_df['Case_No_merged'].str.len().value_counts()

In [ ]:
print('NaN values in case_number:')
print(visas_df['case_number'].isna().sum())
print('NaN values in case_no:')
print(visas_df['case_no'].isna().sum())
print('NaN Values combined:')
print(visas_df['case_number'].isna().sum() + visas_df['case_no'].isna().sum())


In [ ]:
visas_df['Case_No_merged'].str.len()

In [ ]:
visas_df['Case_No_merged'].isna().sum()

CaseNumber is completely filled. No Null Values.

In [ ]:
unique_values_df = visas_df['fw_info_birth_country'].value_counts().rename_axis('unique values').reset_index(name='counts')
unique_values_df
unique_values_df.sort_values(by='counts', ascending = False).plot.bar()

In [ ]:


unique_values_df = visas_df['naics_title'].value_counts().rename_axis('unique values').reset_index(name='counts')
unique_values_df
unique_values_df.sort_values(by='counts', ascending = False).plot.bar()

In [ ]:
unique_values_df = visas_df['case_status'].value_counts().rename_axis('unique values').reset_index(name='counts')
#unique_values_df = visas_df['case_status'].value_counts().rename_axis('unique values').reset_index(level=0, inplace=True)

print(unique_values_df)
unique_values_df.sort_values(by='counts', ascending = False).plot(kind='bar', x='unique values', y='counts', legend=None, title='Ice Cream Sales')




In [ ]:
unique_values_df.plot(kind='pie', labels=unique_values_df['unique values'], y='counts', legend=None, autopct='%.2f%%', title='Ice Cream Sales')
plt.tight_layout()
plt.show()


---
## In welchen US Bundesstasaten ist der Bedarf an Fachkräften am größten?
Die benötigten Spalten sind dafür:
- employer_state

Werfen wir einen Blick in employer_state:



In [ ]:
print(visas_df['employer_state'].value_counts())

print(visas_df['employer_state'].describe(include=[object]))

print(visas_df['employer_state'].head())

It becomes apparent, that abbreviations are used in the mix with the regular state names.
We'll have to translate the values to either only-abbreviation-values or long notations only.

We have found online a table which shows  all abbreviations of the US states: https://docs.omnisci.com/v4.1.1/3_apdx_states.html

Let's read it into a dataframe.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from unicodedata import normalize

table_MN = pd.read_html('https://docs.omnisci.com/v4.1.1/3_apdx_states.html')

In [ ]:
print(f'Total tables: {len(table_MN)}')

pandas found two tables on the specified website

Let's try it once more, but now with a specified keyword.

In [ ]:
table_MN = pd.read_html('https://docs.omnisci.com/v4.1.1/3_apdx_states.html', match='Abbreviation')
print(f'Total tables: {len(table_MN)}')

Jackpot 🎉

Next steps: Convert it to a dataframe.

In [ ]:
states_abbreviations_df = table_MN[0]
states_abbreviations_df.head()

In [ ]:
states_abbreviations_df.info()

We need to convert the State names into Uppercase Values

In [ ]:
states_abbreviations_df['State'] = states_abbreviations_df['State'].str.upper()

Now we'll create a dictionary for all lookup values, so pandas know what it should replace, with which values.
We chose to use replace all names with their abbreviations. So that typos in the state names won't be replaced and we can still analyze them.

In [ ]:
#create a dictionary from the lookup
repl = states_abbreviations_df.set_index('State')['Abbreviation'].to_dict()

In [ ]:
visas_df['employer_state'] = visas_df['employer_state'].replace(repl)

In [ ]:
visas_df['employer_state'].value_counts()

The Data transformation worked 😃, but there's still some work to do.
It seems as some states were not represented in the abbreviation table.
Let's take a closer look.

All abbreviations are two letters long. We have to look out for all values with more than two letters.

In [ ]:

df_temp = pd.DataFrame(visas_df.loc[visas_df['employer_state'].str.len() >2, 'employer_state'])

In [ ]:
df_temp.value_counts()

In [ ]:
del df_temp

The remaining entries are foreign territories of the USA. These have no official abbreviation.
Fortunately there are no Typos in this column, because every other entry could be matched.

Außengebiet der Vereinigten Staaten

Territories of the United States
https://en.wikipedia.org/wiki/Territories_of_the_United_States




We converted the spellings of the US states. Now it's time to visualize the results.
The visualisation will cover 61 unique values, so we'll choose a classic bar chart.


In [ ]:

unique_values_df = visas_df['employer_state'].value_counts().rename_axis('unique values').reset_index(name='counts')
unique_values_df

In [ ]:
unique_values_df.sort_values(by='counts', ascending = False).plot(kind='bar', x='unique values', y='counts', legend=None, title='Count of Employer states', figsize=(15,6))
